In [ ]:
import cartopy.crs as ccrs
from cmocean import cm 
from dino import Experiment
from matplotlib import colors
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import cftime as cft
import xnemogcm as xn
import xgcm

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"
dino_exp = Experiment(path, 'SaltRestoring/restart9')

In [ ]:
T_star = dino_exp.get_T_star()
S_star = dino_exp.get_S_star()

In [ ]:
rho_star = dino_exp.get_rho_star()

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,8), sharey=True)
T_star.isel(x_c=10, t_m=-1, y_c=slice(1,-1)).plot(color='midnightblue', y='gphit', ax=axs[0], label='December')
T_star.isel(x_c=10, t_m=-7, y_c=slice(1,-1)).plot(color='darkred', y='gphit', ax=axs[0], label='June')
S_star.isel(x_c=10).plot(color='black', y='gphit', ax=axs[1])
rho_star.isel(x_c=10, t_m=-1, y_c=slice(1,-1)).plot(color='midnightblue', y='gphit', ax=axs[2], label='December')
rho_star.isel(x_c=10, t_m=-7, y_c=slice(1,-1)).plot(color='darkred', y='gphit', ax=axs[2], label='June')

axs[0].fill_betweenx(dino_exp.domain.gphiu.values[:,0], -2, 32, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[0].margins(x=0, y=0)
axs[0].set_xlabel(r"$T^{*}$ [°C]")
axs[0].set_title('')
axs[0].grid()
axs[0].legend()
#
axs[1].fill_betweenx(dino_exp.domain.gphiu.values[:,0], 34.5, 37.5, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[1].margins(x=0, y=0)
axs[1].set_xlabel(r"$S^{*}$ [psu]")
axs[1].set_title('')
axs[1].grid()
#
axs[2].fill_betweenx(dino_exp.domain.gphiu.values[:,0], 1022, 1028, where=dino_exp.domain.umask.isel(x_f=0, z_c=0)==1, facecolor='grey', alpha=.2)
axs[2].margins(x=0, y=0)
axs[2].set_xlabel(r"$\rho^{*}$ [kg / m3]")
axs[2].set_title('')
axs[2].grid()
axs[2].legend()

## Why Is the density restoring decreasing towards the meridional boundaries? 
    - Problem in equation of state?
        --> EOS seems right, maybe issue in z used as as gdept_0? --> Yes, z is reference pressure level (0 for rhop) 
        --> T restoring on T-points!
    - Find S_star with a density restoring towards Munday (2012) ?
        --> Include denser waters south of channel?

In [ ]:
idx_n = abs(rho_star.gphit - (-40)).argmin()
idx_s = abs(rho_star.gphit - (-65)).argmin()

In [ ]:
from matplotlib.colors import LogNorm

soce = (dino_exp.data.isel(t_y=-1).where(dino_exp.domain.tmask == 1.)).soce.values.ravel()
toce = (dino_exp.data.isel(t_y=-1).where(dino_exp.domain.tmask == 1.)).toce.values.ravel()
vol  = (dino_exp.domain.e1t * dino_exp.domain.e2t * dino_exp.data.isel(t_y=-1).e3t).values.ravel()


fig, ax = plt.subplots()
hb = ax.hexbin(soce, toce,
           C=vol, reduce_C_function=np.sum,
           extent=(34,38,-3,30), gridsize=50, bins='log',
           cmap=cm.matter)
plt.colorbar(hb)
ax.set_ylabel(r'Temperature ($^\circ$C)')
ax.set_xlabel("Salinity (g / kg)")

In [ ]:
rho = dino_exp.get_rho()

In [ ]:
T_star.isel(x_c=10, t_m=-9, y_c=slice(1,-1)).plot(color='darkred', y='gphit')

In [ ]:
rho_star.isel(x_c=10, t_m=-9, y_c=slice(1,-1)).plot(color='darkred', y='gphit')

In [ ]:
rho_star.isel(x_c=10, t_m=-9, y_c=slice(1,-1)).min().values

In [ ]:
def scurve(x, x0, dx):
    """Returns 0 for x<x0 or x>x+dx, and a cubic in between."""
    s = np.minimum(1, np.maximum(0, (x-x0)/dx))
    return (3 - 2*s)*( s*s )
def itau(ytau,taud,ys):
    """Returns a profile tau(ys) that uses s-curves between node,valuesa ytau,taud"""
    taus = 0.*ys
    ks = 0
    for i in range(len(ys)):
        y = ys[i]
        if y>=ytau[ks+1]:
            ks=min(len(ytau)-2,ks+1)
        taus[i] = taud[ks] + ( taud[ks+1] - taud[ks]) * scurve(y, ytau[ks], ytau[ks+1]-ytau[ks])
    return taus
ysteps = np.linspace(-70,70,100)

In [ ]:
yrho, vrho, vrho_jan, vrho_jul   = [-70, 0, 70], [1027.5, 1022.6, 1027.5 -0.4855], [1027.1, 1022.6, 1026.8], [1027.5, 1022.6, 1026.8]
rho_star_c = itau(yrho, vrho, ysteps)
#rho_star_jan = itau(yrho, vrho_jan, ysteps)
#rho_star_jul = itau(yrho, vrho_jul, ysteps)
plt.plot(ysteps, rho_star_c, color='black')
#plt.plot(ysteps, rho_star_jan, color='midnightblue', label='January')
#plt.plot(ysteps, rho_star_jul, color='darkred', label='July')
plt.xlim(-70,70);
plt.ylim(1022,1028);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\rho$');
plt.grid();
plt.legend()
plt.vlines([-65, -45], 1022, 1028, linestyles='dashed', color='black')
plt.hlines([1026.1, 1027.44], -70, 70, linestyles='dashed', color='black')
#plt.hlines([1026.22, 1027.55], -70, 70, linestyles='dashed', color='darkred')

In [ ]:
yt, vt, vt_jan, vt_jul   = [-70, 0, 70], [-0.5, 27., 5.], [0.0, 27., 1.0], [-1.0, 27., 9.0]
t_star = itau(yt, vt, ysteps)
t_star_jan = itau(yt, vt_jan, ysteps)
t_star_jul = itau(yt, vt_jul, ysteps)

plt.plot(ysteps, t_star, color='black')
plt.plot(ysteps, t_star_jan, color='midnightblue', label='January')
plt.plot(ysteps, t_star_jul, color='darkred', label='July')
plt.xlim(-70,70);
plt.ylim(-2,30);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$T^{*}$');
plt.grid();
plt.legend()
plt.vlines([-65, -45], 1022, 1028, linestyles='dashed', color='black')
#plt.hlines([1025.7, 1026.93], -70, 70, linestyles='dashed', color='midnightblue')
#plt.hlines([1026.22, 1027.55], -70, 70, linestyles='dashed', color='darkred')

In [ ]:
yrho,rhod   = [-70, -65, 0, 70], [1027.3, 1027.3, 1022.6, 1026.8]
yt,td       = [-70, 0, 70], [-0.5, 27., 5]
yt_d,td_d       = [-70, -10, 70], [0, 27., 0.]
yt_j,td_j       = [-70, 10, 70], [-1, 27., 10]
ys, sd      = [-70, -65, -15, 0, 15, 70 ], [35.2, 35.2, 37., 36., 37., 35.3]
ysteps = np.linspace(-70,70,100)
rho_star = itau(yrho, rhod, ysteps)
t_star = itau(yt, td, ysteps)
s_star = itau(ys, sd, ysteps)
t_star_d = itau(yt_d, td_d, ysteps)
t_star_j = itau(yt_j, td_j, ysteps)
plt.plot(yrho,rhod,'x')
plt.plot(ysteps, rho_star)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\tau$ (Pa)');
plt.grid();

In [ ]:
plt.plot(yt,td,'x')
plt.plot(ysteps, t_star)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\tau$ (Pa)');
plt.grid();

In [ ]:
plt.plot(ys,sd,'x')
plt.plot(ysteps, s_star)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\tau$ (Pa)');
plt.grid();

In [ ]:
nml = dino_exp.namelist['nameos']
s_star_2 = ( rho - 1026.
    + nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( t - 10.)) * ( t - 10.)
) / nml['rn_b0'] + 35.

In [ ]:
plt.plot(ysteps, s_star_2)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\tau$ (Pa)');
plt.grid();

In [ ]:
nml = dino_exp.namelist['nameos']
rho_star_2 = ( 
    - nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( t_star - 10.)) * ( t_star - 10.)
    + nml['rn_b0'] * (s_star - 35.) + 1026. 
)

In [ ]:
plt.plot(ysteps, rho_star_2)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\tau$ (Pa)');
plt.grid();

In [ ]:
nml = dino_exp.namelist['nameos']
rho_star_d = ( 
    - nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( t_star_d - 10.)) * ( t_star_d - 10.)
    + nml['rn_b0'] * (s_star - 35.) + 1026. 
)

In [ ]:
plt.plot(ysteps, rho_star_d)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\tau$ (Pa)');
plt.grid();

In [ ]:
rho_star_j = ( 
    - nml['rn_a0'] * (1. + 0.5 * nml['rn_lambda1'] * ( t_star_j - 10.)) * ( t_star_j - 10.)
    + nml['rn_b0'] * (s_star - 35.) + 1026. 
)

In [ ]:
plt.plot(ysteps, rho_star_j)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\tau$ (Pa)');
plt.grid();

In [ ]:
nml = dino_exp.namelist['nameos']

In [ ]:
nml

In [ ]:
def T_n_star(T_s_star, drho, pm=1.0):
    """ 
    Computing the Northern temperature restoring depending on:
        - the Southern temperature restoring T_s_star
        - the desired difference in density restoring drho
    Assuming symmetric salt restoring.    
    """
    lambda_1    = 0.06
    a0          = 0.165
    # Square-root of quadratic:
    a           = 0.5 * lambda_1
    b           = (1 - 10 * lambda_1)
    c           = drho / a0 - a * T_s_star ** 2 - b * T_s_star
    return(
        (-b + pm * np.sqrt(b**2 - 4 * a * c)) / 2 / a 
    )

In [ ]:
T_s_star = np.linspace(-1,1,100)

In [ ]:
T_n = T_n_star(T_s_star=T_s_star, drho=-0.4855, pm=1)

In [ ]:
plt.plot(T_s_star, T_n)
plt.xlim(-1,1);
plt.xlabel(r'$T_{s}^{*}$');
plt.ylabel(r'$T_{n}^{*}$');
plt.grid();

In [ ]:
def S_star(T_star, rho_star):
    """ 
    Computing the salinity restoring depending on:
        - temperature restoring
        - density restoring
    Assuming symmetric salt restoring.    
    """
    lambda_1    = 0.06
    a0          = 0.165
    S           = (
        rho_star - 1026.
        + a0 * (1 + 0.5 * lambda_1 * (T_star - 10.)) * (T_star - 10.)
    ) / 0.76554 + 35.
    return(S)

def rho_star(T_star, S_star):
    """ 
    Computing the density restoring depending on:
        - temperature restoring
        - salinity restoring
    Assuming symmetric salt restoring.    
    """
    lambda_1    = 0.06
    a0          = 0.165
    b0          = 0.76554
    rho         = (
        - a0 * (1 + 0.5 * lambda_1 * (T_star - 10.)) * (T_star - 10.)
        + b0 * (S_star - 35.)
        + 1026.
    ) 
    return(rho)

In [ ]:
S_star_c = S_star(T_star=t_star, rho_star=rho_star_c)

In [ ]:
plt.plot(ysteps, S_star_c)
plt.plot(ysteps, s_nodes)
plt.xlim(-70,70);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'Salinity (p)');
plt.grid();

In [ ]:
S_star_c

In [ ]:
ys, vs   = [-70, -40, 0, 40,  70], [35.401, 34.505, 36.09, 34.931, 35.401]
s_nodes = itau(ys, vs, ysteps)

In [ ]:
rho_jul_star = rho_star(t_star_jul, s_nodes)
rho_jan_star = rho_star(t_star_jan, s_nodes)

In [ ]:
yrho, vrho, vrho_jan, vrho_jul   = [-70, 0, 70], [1027.5, 1022.6, 1027.5 -0.4855], [rho_jan_s_star, 1022.6, 1027.5 -0.4855], [rho_jul_s_star, 1022.6, rho_jul_s_star]
rho_star_c = itau(yrho, vrho, ysteps)
#rho_star_jan = itau(yrho, vrho_jan, ysteps)
#rho_star_jul = itau(yrho, vrho_jul, ysteps)
plt.plot(ysteps, rho_star_c, color='black')
plt.plot(ysteps, rho_jan_star, color='midnightblue', label='January')
plt.plot(ysteps, rho_jul_star, color='darkred', label='July')
plt.xlim(-70,70);
plt.ylim(1022,1028);
plt.xlabel('Latitude ($^\circ$N)');
plt.ylabel(r'$\rho$');
plt.grid();
plt.legend()
plt.vlines([-65, -45], 1022, 1028, linestyles='dashed', color='black')
plt.hlines([1026.1, 1027.44], -70, 70, linestyles='dashed', color='black')
#plt.hlines([1026.22, 1027.55], -70, 70, linestyles='dashed', color='darkred')

In [ ]:
rho_jan_n_star = rho_star(9.0, S_s_star)

In [ ]:
rho_jan_n_star

In [ ]:
rho_jul_star